<a href="https://colab.research.google.com/github/imabari/ImabariScraping/blob/master/%E6%B2%96%E7%B8%84%E5%B8%82%E5%A0%B4%E6%9C%88%E5%A0%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pdfplumber

     |████████████████████████████████| 51kB 4.0MB/s 
     |████████████████████████████████| 5.6MB 13.9MB/s 
     |████████████████████████████████| 143kB 42.1MB/s 
     |████████████████████████████████| 1.9MB 40.5MB/s 
  Created wheel for pdfplumber: filename=pdfplumber-0.5.25-cp36-none-any.whl size=31560 sha256=9e4d208172158ec380dac8c280a10bd02bd145b1e536ddd4fecea02586aa369e
  Stored in directory: /root/.cache/pip/wheels/fc/bd/c8/a28e0008387d034112de3c7c3bf6dbefa25653909a6269e4ff
Successfully built pdfplumber


In [2]:
import pathlib

In [3]:
import pandas as pd
import pdfplumber
import requests

In [4]:
def fetch_file(url, dir="."):

    r = requests.get(url)
    r.raise_for_status()

    p = pathlib.Path(dir, pathlib.PurePath(url).name)
    p.parent.mkdir(parents=True, exist_ok=True)

    with p.open(mode="wb") as fw:
        fw.write(r.content)
    return p

In [5]:
table_settings = {

    # 垂直基準
    "vertical_strategy": "explicit",
    "explicit_vertical_lines": [80, 180, 235, 265, 325, 355, 410, 435, 490, 530],

    # 水平基準
    "horizontal_strategy": "text",
   
    # テキストの左端と右端が垂直線と完全に一致していない場合の許容値
    "intersection_tolerance": 30,
}

In [6]:
url = "https://www.pref.okinawa.jp/site/norin/oroshiuri/documents/documents/geppou3104.pdf"

In [7]:
pdf_path = fetch_file(url)

In [8]:
with pdfplumber.open(pdf_path) as pdf:

    dfs = []

    for page in pdf.pages[6:12]:

        crop = page.within_bbox((0, 105, page.width, 800))

        table = crop.extract_table(table_settings)

        df_tmp = pd.DataFrame(table)

        for col in df_tmp.select_dtypes(include=object).columns:
            df_tmp[col] = df_tmp[col].str.replace(",", "").str.strip()

        df_tmp = df_tmp.mask(df_tmp == "").dropna(thresh=2)

        df_tmp = df_tmp[df_tmp[0] != "品　目"]

        # print(df_tmp.shape)

        dfs.append(df_tmp)

In [9]:
df = pd.concat(dfs)

In [10]:
df

,0,1,2,3,4,5,6,7,8
1,だいこん,6442,NaN,254596,NaN,200,NaN,261238,NaN
2,NaN,114247,18,20438370,80,86400,432,20639017,79
3,かぶ,87,NaN,333,NaN,NaN,NaN,420,NaN
4,NaN,9720,112,114725,345,NaN,NaN,124445,296
5,にんじん,144230,NaN,208700,NaN,32150,NaN,385080,NaN
...,...,...,...,...,...,...,...,...,...
15,NaN,2644596,2784,6480,3240,NaN,NaN,2651076,2785
16,果実加工,NaN,NaN,NaN,NaN,400,NaN,400,NaN
17,NaN,NaN,NaN,NaN,NaN,216000,540,216000,540
18,セミノール,NaN,NaN,845,NaN,12579,NaN,13424,NaN


In [11]:
df[0] = df[0].fillna(method="ffill")

In [12]:
df_even = (
    df[::2]
    .rename(
        columns={
            0: "品名",
            1: "県内_数量",
            2: "県内_単価",
            3: "県外_数量",
            4: "県外_単価",
            5: "外国_数量",
            6: "外国_単価",
            7: "総計_数量",
            8: "総計_単価",
        }
    )
    .dropna(how="all", axis=1)
)

In [13]:
df_odd = (
    df[1::2]
    .rename(
        columns={
            0: "品名",
            1: "県内_金額",
            2: "県内_単価",
            3: "県外_金額",
            4: "県外_単価",
            5: "外国_金額",
            6: "外国_単価",
            7: "総計_金額",
            8: "総計_単価",
        }
    )
    .dropna(how="all", axis=1)
)

In [14]:
df1 = pd.merge(df_even, df_odd, on="品名").set_index("品名")

In [15]:
df1

,県内_数量,県外_数量,外国_数量,総計_数量,県内_金額,県内_単価,県外_金額,県外_単価,外国_金額,外国_単価,総計_金額,総計_単価
品名,,,,,,,,,,,,
だいこん,6442,254596,200,261238,114247,18,20438370,80,86400,432,20639017,79
かぶ,87,333,NaN,420,9720,112,114725,345,NaN,NaN,124445,296
にんじん,144230,208700,32150,385080,13356377,93,22004529,105,3165912,98,38526818,100
ごぼう,301,1820,22210,24331,109242,363,732645,403,4658796,210,5500683,226
たけのこ,NaN,78,6,84,NaN,NaN,58137,745,6804,1134,64941,773
...,...,...,...,...,...,...,...,...,...,...,...,...
パッションフル－ツ,5866,NaN,NaN,5866,9308086,1587,NaN,NaN,NaN,NaN,9308086,1587
ドラゴンフル－ツ,NaN,NaN,1550,1550,NaN,NaN,NaN,NaN,926910,598,926910,598
アテモヤ,950,2,NaN,952,2644596,2784,6480,3240,NaN,NaN,2651076,2785


In [16]:
df1.to_csv("okinawa.csv", encoding="utf_8_sig")

In [17]:
from google.colab import files

files.download("okinawa.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>